# **Measuring Bias in multiclass classification**


This tutorial will explain how to measure bias in a multiclass classification task using the holisticai library. We will introduce here some of the functions that can help study algorithmic bias.

The sections are organised as follows :
1. Load the data : we load the student dataset as a pandas DataFrame
2. Train a Model : we train a model (sklearn)
3. Measure Efficacy : we compute a few efficacy metrics.

## **Load the data**

In [1]:
# Base imports
import sys
sys.path.append('../../')
import numpy as np
import pandas as pd

The student dataset can be easily

In [2]:
from holisticai.datasets import load_student

# load data
df = load_student()['frame']

# Make data multiclass by slicing into 4 buckets
y = df['G3'].to_numpy()
buckets = np.array([8, 11, 14])
y_cat = (y.reshape(-1, 1) > buckets.reshape(1, -1)).sum(axis=1)
df['target'] = y_cat

# map dictionary
grade_dict = {0:'very-low', 1:'low', 2:'high',3:'very-high'}
df['target'] = df['target'].map(grade_dict)

# drop the other grade columns
df = df.drop(columns=['G1','G2','G3'])

df

c:\Users\Cristian\anaconda3\envs\metrics\lib\site-packages\sklearn\datasets\_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,target
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,no,no,4,3,4,1,1,3,6,very-low
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,no,5,3,3,1,1,3,4,very-low
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,no,4,3,2,2,3,3,10,low
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,3,2,2,1,1,5,2,very-high
4,GP,F,16,U,GT3,T,3,3,other,other,...,no,no,4,3,2,1,2,5,4,low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2,2,services,services,...,no,no,5,5,4,4,5,4,11,low
391,MS,M,17,U,LE3,T,3,1,services,services,...,yes,no,2,4,5,3,4,2,3,very-high
392,MS,M,21,R,GT3,T,1,1,other,other,...,no,no,5,5,3,3,3,3,3,very-low
393,MS,M,18,R,LE3,T,3,2,services,other,...,yes,no,4,4,1,3,4,5,0,low


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

In [4]:
# preprocess categorical columns
def preprocess_data(data, protected_attributes, label_attributes):
    categoricals = []
    for col in data.columns:
        if col not in protected_attributes and col not in label_attributes and data[col].dtype == object:
            categoricals.append(col)

    cat_encoder = OneHotEncoder()  
    enc = cat_encoder.fit_transform(data[categoricals])
    enc = pd.DataFrame(enc.toarray())
    df = pd.concat([data,enc],axis=1).drop(columns=categoricals) # add encoded columns
    df = df.rename(str, axis='columns')
    return df

In [5]:
# we don't want to encode protected attributes
protected_attributes = ['sex', 'address', 'Mjob', 'Fjob']
label_attributes = ['target']

# Load, preprocess and split for training
preproc_data = preprocess_data(df, protected_attributes, label_attributes)
train, test = train_test_split(preproc_data, test_size=0.4, random_state=42)

# display
preproc_data

,sex,age,address,Medu,Fedu,Mjob,Fjob,traveltime,studytime,failures,...,19,20,21,22,23,24,25,26,27,28
0,F,18,U,4,4,at_home,teacher,2,2,0,...,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
1,F,17,U,1,1,at_home,other,1,2,0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,F,15,U,1,1,at_home,other,1,2,3,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
3,F,15,U,4,2,health,services,1,3,0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
4,F,16,U,3,3,other,other,1,2,0,...,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,M,20,U,2,2,services,services,1,2,2,...,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
391,M,17,U,3,1,services,services,2,1,0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
392,M,21,R,1,1,other,other,1,1,3,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
393,M,18,R,3,2,services,other,3,1,0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


## **2. Train a Model**

In [6]:
# set up data for training
X_train = train.drop(columns=label_attributes)
X_test = test.drop(columns=label_attributes)
y_train = train['target']
y_test = test['target']

# Train a simple Random Forest Classifier
model = RandomForestClassifier(random_state=111)
model.fit(X_train.drop(columns=protected_attributes), y_train)

# Predict values
y_pred = model.predict(X_test.drop(columns=protected_attributes))
y_proba = model.predict_proba(X_test.drop(columns=protected_attributes))

## **3. Measure Efficacy**

In [7]:
from holisticai.efficacy.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, accuracy_score

In [8]:
confusion_matrix(y_pred, y_test, classes=['very-low','low','high','very-high'], normalize='true')

,very-low,low,high,very-high
very-low,0.382979,0.085106,0.030303,0.064516
low,0.404255,0.680851,0.666667,0.322581
high,0.148936,0.170213,0.242424,0.354839
very-high,0.063830,0.063830,0.060606,0.258065


In [9]:
from holisticai.efficacy.metrics import multiclassification_efficacy_metrics

In [10]:
multiclassification_efficacy_metrics(y_pred, y_test, by_class=True)

,Value,high,low,very-high,very-low,Reference
Metric,,,,,,
Accuracy,0.417722,,,,,1
Balanced accuracy,0.391080,,,,,1
Precision,0.417722,0.235294,0.385542,0.5,0.72,1
Recall,0.417722,0.242424,0.680851,0.258065,0.382979,1
F1-Score,0.417722,0.238806,0.492308,0.340426,0.5,1


In [11]:
multiclassification_efficacy_metrics(y_pred, y_test, y_proba, classes=['very-low','low','high','very-high'], by_class=True)

,Value,very-low,low,high,very-high,Reference
Metric,,,,,,
Accuracy,0.417722,,,,,1
Balanced accuracy,0.391080,,,,,1
Precision,0.417722,0.72,0.385542,0.235294,0.5,1
Recall,0.417722,0.382979,0.680851,0.242424,0.258065,1
F1-Score,0.417722,0.5,0.492308,0.238806,0.340426,1
AUC,0.437751,0.312728,0.58894,0.47697,0.372365,1
Log Loss,1.561233,,,,,0
